In [ ]:
import numpy as np

In [ ]:
from dolfinx.mesh import compute_boundary_facets, create_unit_cube, locate_entities_boundary
from dolfinx.io import XDMFFile
import dolfinx.fem as fem
from ufl import grad, dot, inner, Identity, TestFunction, TrialFunction, tr, ln, det, jump, Measure, FacetArea, avg, as_vector
from mpi4py import MPI
from petsc4py.PETSc import ScalarType as st

In [ ]:
from ufl import derivative, diff, variable

In [ ]:
from dolfinx.nls import NewtonSolver

In [ ]:
comm = MPI.COMM_WORLD
mesh = create_unit_cube(comm, 5, 5, 5)
V = fem.VectorFunctionSpace(mesh, ("CR", 1))
Vp = fem.VectorFunctionSpace(mesh, ("CG", 1))
u = fem.Function(V)
du, v = TrialFunction(V), TestFunction(V)

In [ ]:
F = Identity(len(u)) + grad(u)
I = inner(F, F)
J = det(F)

In [ ]:
mu, lmbda = fem.Constant(mesh, st(1.0)), fem.Constant(mesh, st(1.e3))

In [ ]:
psi = mu/2. * (I - 3) - mu * ln(J) + lmbda/2 * (J - 1)**2

In [ ]:
md = {"quadrature_degree":5}
dx = Measure("dx", metadata=md)
dS = Measure("dS", metadata=md)

In [ ]:
q = mu*3.0
h_avg = avg(FacetArea(mesh))

In [ ]:
omega = fem.Constant(mesh, st(1e-6))

In [ ]:
dpsi_du = derivative(psi, u, v) * dx + q/h_avg * dot(jump(u), jump(v)) * dS

In [ ]:
d2psi_du2 = derivative(dpsi_du, u, du)

In [ ]:
stabilization = omega * inner(grad(du), grad(v)) * dx

In [ ]:
left = lambda x: np.isclose(x[0], 0)
bottom = lambda x: np.isclose(x[1], 0)
back = lambda x: np.isclose(x[2], 0)
right = lambda x: np.isclose(x[1], 1.)
left_facet = locate_entities_boundary(mesh, mesh.topology.dim-1, left)
bottom_facet = locate_entities_boundary(mesh, mesh.topology.dim-1, bottom)
right_facet = locate_entities_boundary(mesh, mesh.topology.dim-1, right)
back_facet = locate_entities_boundary(mesh, mesh.topology.dim-1, back)

In [ ]:
fdim = mesh.topology.dim-1

In [ ]:
left_dofs_x = fem.locate_dofs_topological(V.sub(0), fdim, left_facet)

In [ ]:
right_dofs_x = fem.locate_dofs_topological(V.sub(0), fdim, right_facet)

In [ ]:
bottom_dofs_y = fem.locate_dofs_topological(V.sub(1), fdim, bottom_facet)

In [ ]:
back_dofs_z = fem.locate_dofs_topological(V.sub(2), fdim, back_facet)

In [ ]:
bc_right_val = fem.Constant(mesh, st(0.001))
bcx = fem.dirichletbc(st(0), left_dofs_x, V.sub(0))
bcy = fem.dirichletbc(st(0), bottom_dofs_y, V.sub(1))
bcz = fem.dirichletbc(st(0), back_dofs_z, V.sub(2))
bcx_right = fem.dirichletbc(bc_right_val, right_dofs_x, V.sub(0))

In [ ]:
bc_right_val.value

In [ ]:
bcs=[bcx, bcy, bcz, bcx_right]

In [ ]:
problem = fem.NonlinearProblem(dpsi_du, u, bcs=bcs, J=d2psi_du2+stabilization)

In [ ]:
solver = NewtonSolver(comm, problem)

```
Help on NewtonSolver in module dolfinx.nls object:

class NewtonSolver(dolfinx.cpp.nls.NewtonSolver)
 |  NewtonSolver(comm: 'MPI.Intracomm', problem: 'NonlinearProblem')
 |  
 |  Method resolution order:
 |      NewtonSolver
 |      dolfinx.cpp.nls.NewtonSolver
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, comm: 'MPI.Intracomm', problem: 'NonlinearProblem')
 |      A Newton solver for non-linear problems.
 |  
 |  setP(self, P: 'types.FunctionType', Pmat: 'PETSc.Mat')
 |      Set the function for computing the preconditioner matrix
 |      
 |      Args:
 |          P: Function to compute the preconditioner matrix
 |          Pmat: Matrix to assemble the preconditioner into
 |  
 |  solve(self, u: 'fem.Function')
 |      Solve non-linear problem into function u. Returns the number
 |      of iterations and if the solver converged.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  A
 |      Jacobian matrix
 |  
 |  b
 |      Residual vector
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from dolfinx.cpp.nls.NewtonSolver:
 |  
 |  setF(...)
 |      setF(self: dolfinx.cpp.nls.NewtonSolver, arg0: Callable[[vec, vec], None], arg1: vec) -> None
 |  
 |  setJ(...)
 |      setJ(self: dolfinx.cpp.nls.NewtonSolver, arg0: Callable[[vec, mat], None], arg1: mat) -> None
 |  
 |  set_form(...)
 |      set_form(self: dolfinx.cpp.nls.NewtonSolver, arg0: Callable[[vec], None]) -> None
 |  
 |  set_update(...)
 |      set_update(self: dolfinx.cpp.nls.NewtonSolver, arg0: Callable[[dolfinx.cpp.nls.NewtonSolver, vec, vec], None]) -> None
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties inherited from dolfinx.cpp.nls.NewtonSolver:
 |  
 |  krylov_solver
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from dolfinx.cpp.nls.NewtonSolver:
 |  
 |  atol
 |      Absolute tolerance
 |  
 |  convergence_criterion
 |      Convergence criterion, either 'residual' (default) or 'incremental'
 |  
 |  error_on_nonconvergence
 |  
 |  max_it
 |      Maximum number of iterations
 |  
 |  relaxation_parameter
 |      Relaxation parameter
 |  
 |  report
 |  
 |  rtol
 |      Relative tolerance
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and return a new object.  See help(type) for accurate signature.
```

In [ ]:
solver.atol=1.e-8
solver.rtol=1.e-4
solver.convergence_criterion='incremental'
solver.max_it=500
solver.report=True

In [ ]:
num_steps = 100
loads = np.linspace(0, 1, num_steps)
for load in loads:
    bc_right_val.value=load
    try:
        num_its, converged = solver.solve(u)
    except:
        omega.value *= 10
        print(f"convergence failed, trying gradient flow, with omega = {omega.value[0]:.5e}")
    while not(converged):
        try:
            num_its, converged = solver.solve(u)
            print(f"omega={omega:.5e}")
        except:
            omega.value *= 10.
            print(f"convergence failed, trying gradient flow, with omega = {omega.value[0]:.5e}")
    print(f"load: {load}, num_iteration: {num_its}, convergence: {converged}")

In [ ]:
help(solver)